In [24]:
import numpy as np
from numpy import radians, pi
from lib.cartpolesystem_new import CartPoleSystem, Cart, Pole, DCMotor
from lib.cartpoleenv_new import CartPoleEnv
from lib.colors import Colors
from lib.controllers_new import LQR
import time
from time import perf_counter
from matplotlib import pyplot as plt
from scipy.io import savemat
from lib.numerical import fe_step, rk4_step

In [25]:
dt = 0.01
g = 9.81

cart = Cart(0.5, 0.005, (-0.8, 0.8))
motor = DCMotor(0.02, 0.02, 0.2, 0.05, 8.7e-5, 8.7e-5, (-24, 24), (-100, 100))
poles = [Pole(1, 0.2, 0.1), Pole(0.5, 0.2, 0.1)]
n = len(poles)

system = CartPoleSystem(cart, motor, poles, g)

max_time = 10
N = int(max_time/dt)

env = CartPoleEnv(system, dt, N, rk4_step)
env.observation_space.shape

(6,)

In [26]:
linearization_x0 = np.array([0, 0, radians(0), 0, radians(0), 0])
linearization_u0 = np.array([0])

A0, B0 = system.linearize(linearization_x0, linearization_u0)

C = np.eye(2+2*n)
D = np.zeros((2+2*n, 1))

ct_dt = dt

lqr = LQR(ct_dt)

Q = np.eye(2+2*n)
R = np.diag([0.01])

A_d0, B_d0 = lqr.discretize(A0, B0, C, D)
K_d0 = lqr.calculate_K_lqr_d(A_d0, B_d0, Q, R)

In [27]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print("A")
print(A0)
print("B")
print(B0)

A
[[ 0.000  1.000  0.000  0.000  0.000  0.000]
 [ 0.000 -0.603 -3.020  0.308 -1.510  0.308]
 [ 0.000  0.000  0.000  1.000  0.000  0.000]
 [ 0.000  2.586  54.988 -5.605  6.472 -1.320]
 [ 0.000  0.000  0.000  0.000  0.000  1.000]
 [ 0.000  2.586  12.945 -1.320  48.515 -9.891]]
B
[[ 0.000]
 [ 1.437]
 [ 0.000]
 [-6.158]
 [ 0.000]
 [-6.158]]


In [28]:
def is_controllable(A, B):
    n = A.shape[0]  # Number of states
    controllability_matrix = np.column_stack([np.linalg.matrix_power(A, i) @ B for i in range(n)])
    rank = np.linalg.matrix_rank(controllability_matrix)
    return rank == n

if is_controllable(A0, B0):
    print("The system is controllable.")
else:
    print("The system is not controllable.")

The system is controllable.


In [29]:
x0 = np.array([-0.1, 0, radians(-1), 0, radians(1), 0])

state, _ = env.reset(x0)

r = np.array([0.1, 0] + [radians(0), 0] + [radians(0), 0])

last_update = perf_counter()

for i in range(N-1):
    
    while perf_counter() < last_update + dt:
        pass
    last_update = perf_counter()
    error = system.calculate_error(state, r)

    control = lqr.feedback(K_d0, error)
    time1 = time.perf_counter()
    state, reward, done, msg, _ = env.step(control)
    time2 = time.perf_counter()
    env.render()
env.close()
print(time2-time1)

AttributeError: 'NoneType' object has no attribute 'fill'